In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils, time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

model = load_model('face_recognition.h5')

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame =video_capture.read()
    if not ret:
        print(':(')
        break
        
    res_frame = cv2.resize(frame, (frame.shape[1] // 4, frame.shape[0] // 4))
    faces = face_cascade.detectMultiScale(res_frame)
    for f in faces:
        start_time = time.time()
        (x, y, w, h) = [v * 4 for v in f] 
        face_img = frame[y:y+h, x:x+w]
        resized = cv2.resize(face_img,(224,224))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)
        result = np.argmax(result, axis=1)
#         actual.append(true_act)
        
        if result >= 0.8:
            prediction = 'Tidak pakai masker'
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(frame, prediction, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)
        else:
            prediction = 'Pakai masker'
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(frame, prediction, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
        
        runpredict = round((time.time() - start_time), 3)
#         sumRun = sumRun + runpredict;
#         count += 1

    cv2.imshow('Face mask detector', frame)

    if cv2.waitKey(1) == 27:
        break
    if cv2.getWindowProperty('Face mask detector',cv2.WND_PROP_VISIBLE) < 1:        
        break
        
video_capture.release()
cv2.destroyAllWindows()